# This notebook is for course Five : Regression (Supervised Learning)

In [95]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Importing Data Files

In [96]:
#Import Product DataSet here
product_data = pd.read_csv('Product Data Set - Student 2 of 3.csv',sep='|')
product_data.head()

,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE
0,30001,HEALTH & BEAUTY,$7.45
1,30002,HEALTH & BEAUTY,$5.35
2,30003,HEALTH & BEAUTY,$5.49
3,30004,HEALTH & BEAUTY,$6.46
4,30005,HEALTH & BEAUTY,$7.33


In [97]:
#Import Transaction DataSet Here
transactions_data = pd.read_csv('Transaction Data Set - Student 3 of 3.csv',sep='|')
transactions_data.head()

,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT
0,10114,30011,4,0.0,1/2/2015,0
1,10217,30016,3,0.0,1/2/2015,0
2,10224,30013,4,0.0,1/2/2015,0
3,10103,30012,3,0.2,1/2/2015,0
4,10037,30010,8,0.0,1/2/2015,0


In [98]:
#Import Customer Dataset Here
customer_data=pd.read_csv('Customer Data Set - Student 1 of 3.csv')
customer_data.head()


,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS
0,10001,0,64,"$133,498",5,enrolled,06-03-2013,4,Single
1,10002,0,42,"$94,475",9,notenrolled,NaN,6,Married
2,10003,0,40,"$88,610",9,enrolled,02-09-2010,5,Married
3,10004,0,38,"$84,313",8,enrolled,06-04-2015,1,Single
4,10005,0,30,"$51,498",3,notenrolled,NaN,1,Single


### Changing Data Types

In [99]:
customer_data['INCOME']=customer_data['INCOME'].map(lambda x : x.replace('$',''))

In [ ]:
customer_data['INCOME']=customer_data['INCOME'].map(lambda x : int(x.replace(',','')))

### Creating Customer View

In [ ]:
trans_products=transactions_data.merge(product_data,how='inner', left_on='PRODUCT NUM', right_on='PRODUCT CODE')

In [ ]:
trans_products['UNIT LIST PRICE']=trans_products['UNIT LIST PRICE'].map(lambda x : float(x.replace('$','')))

In [ ]:
trans_products['Total_Price']=trans_products['QUANTITY PURCHASED'] * trans_products['UNIT LIST PRICE'] * (1- trans_products['DISCOUNT TAKEN'])

In [ ]:
customer_prod_categ=trans_products.groupby(['CUSTOMER NUM','PRODUCT CATEGORY']).agg({'Total_Price':'sum'})

In [ ]:
customer_prod_categ=customer_prod_categ.reset_index()

In [ ]:
customer_pivot=customer_prod_categ.pivot(index='CUSTOMER NUM',columns='PRODUCT CATEGORY',values='Total_Price')

In [ ]:
trans_total_spend=trans_products.groupby('CUSTOMER NUM').agg({'Total_Price':'sum'}).\
rename(columns={'Total_Price':'TOTAL SPENT'})

In [ ]:
customer_KPIs=customer_pivot.merge(trans_total_spend,how='inner',left_index=True, right_index=True )

In [ ]:
customer_KPIs=customer_KPIs.fillna(0)


In [ ]:
customer_all_view=customer_data.merge(customer_KPIs,how='inner', left_on='CUSTOMERID', right_index=True)

In [ ]:
customer_all_view.head()

,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,TOTAL SPENT
0,10001,0,64,133498,5,enrolled,06-03-2013,4,Single,4022.430,1601.315,68.688,1134.337,6826.770
1,10002,0,42,94475,9,notenrolled,NaN,6,Married,2312.509,2473.163,276.779,0.000,5062.451
2,10003,0,40,88610,9,enrolled,02-09-2010,5,Married,2887.382,5414.418,260.640,0.000,8562.440
3,10004,0,38,84313,8,enrolled,06-04-2015,1,Single,3637.213,1840.211,45.270,0.000,5522.694
4,10005,0,30,51498,3,notenrolled,NaN,1,Single,213.512,0.000,0.000,0.000,213.512


# Linear Regression 

In [ ]:
#Feed the following variables to the linear regression model to predict the total spending
DF_input=customer_all_view[['GENDER','AGE','INCOME','EXPERIENCE SCORE','LOYALTY GROUP','HOUSEHOLD SIZE','MARITAL STATUS']]
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,enrolled,4,Single
1,0,42,94475,9,notenrolled,6,Married
2,0,40,88610,9,enrolled,5,Married
3,0,38,84313,8,enrolled,1,Single
4,0,30,51498,3,notenrolled,1,Single


In [ ]:
#Create four columns that are named Single, Married, Divorced, and Widow/Widower.
DF_input["MARITAL STATUS"].unique()
pd.get_dummies(DF_input["MARITAL STATUS"]).head(10)

,Divorced,Married,Single,Widow/Widower
0,0,0,1,0
1,0,1,0,0
2,0,1,0,0
3,0,0,1,0
4,0,0,1,0
5,0,1,0,0
6,0,1,0,0
7,0,0,1,0
8,0,1,0,0
9,0,1,0,0


In [ ]:
#binary encode the loyalty variable
def encode_loyalty(value):
    if value == "enrolled" :
        return 1 
    else : 
        return 0

In [ ]:
#Apply this function to the “loyalty group” column
DF_input['LOYALTY GROUP']=DF_input['LOYALTY GROUP'].apply(encode_loyalty)
DF_input.head()

/tmp/ipykernel_59001/2399072271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_input['LOYALTY GROUP']=DF_input['LOYALTY GROUP'].apply(encode_loyalty)


,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,0,4,Single
1,0,42,94475,9,0,6,Married
2,0,40,88610,9,0,5,Married
3,0,38,84313,8,0,1,Single
4,0,30,51498,3,0,1,Single


In [ ]:
#replace the “DF_input” frame with the one-hot encoded Pandas data frame
DF_input=pd.get_dummies(DF_input)
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0,64,133498,5,0,4,0,0,1,0
1,0,42,94475,9,0,6,0,1,0,0
2,0,40,88610,9,0,5,0,1,0,0
3,0,38,84313,8,0,1,0,0,1,0
4,0,30,51498,3,0,1,0,0,1,0


In [ ]:
#re-create the Pandas data frame for the NumPy format
DF_input_column_names = DF_input.columns.values
DF_input_np=preprocessing.minmax_scale(DF_input)
Reg_input_scaled=pd.DataFrame(DF_input_np, columns=DF_input_column_names)
Reg_input_scaled=pd.DataFrame(DF_input_np, columns=DF_input_column_names)
Reg_input_scaled.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0.0,0.638889,0.872818,0.444444,0.0,0.6,0.0,0.0,1.0,0.0
1,0.0,0.333333,0.572046,0.888889,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.305556,0.526842,0.888889,0.0,0.8,0.0,1.0,0.0,0.0
3,0.0,0.277778,0.493722,0.777778,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.166667,0.240799,0.222222,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
#Create a train – test split of 80% training and 20% testing
X_train,X_test,Y_train,Y_test = train_test_split(Reg_input_scaled,customer_all_view['TOTAL SPENT'], test_size=0.2,random_state=42)

In [ ]:
#Train the model on the training data set
regr = LinearRegression()
regr.fit(X_train, Y_train)

LinearRegression()

In [ ]:
#predict the values of total spending for the testing feature and find coefficients
Y_pred = regr.predict(X_test)
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [ 6.36174708e+01 -7.20002849e+03  1.28420193e+04  1.48781961e+03
  1.81898940e-12  8.47620685e+02  2.81862333e+02  2.13401801e+01
  3.42083084e+01 -3.37410822e+02]


In [ ]:
#order of coefficients
Reg_input_scaled.columns.values

array(['GENDER', 'AGE', 'INCOME', 'EXPERIENCE SCORE', 'LOYALTY GROUP',
       'HOUSEHOLD SIZE', 'MARITAL STATUS_Divorced',
       'MARITAL STATUS_Married', 'MARITAL STATUS_Single',
       'MARITAL STATUS_Widow/Widower'], dtype=object)

In [ ]:
#Obtain the intercept
print('intercept: \n', regr.intercept_)

intercept: 
 3.9149491787711668


In [ ]:
#Evaluate the performance of the linear model by calculating the mean squared error.
print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))

Mean squared error: 3456747.93
